In [2]:
import os

import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LSTM, Dropout, Bidirectional

from sklearn.utils import shuffle
import numpy as np

from myModel import Mymodel

print(tf.__version__)

2.2.0


In [3]:
vocab_size=5000 , 
max_sequence_length=25,
embedding_dim = 256,
number_lstm_units = 256,
number_dense_units = 256,
rate_drop_lstm = 0.17 , 
rate_drop_dense = 0.01
df =pd.read_csv('./dataset/dictonary.csv')
imgDir='D:\TF\image-Text-similarity\dataset\Triplet'

In [4]:
from sklearn.model_selection import train_test_split
dictonary = []
labels = []
imagePath = []
dataset = []
for index,row in df.iterrows():
    labels.append(int(row['id']))
    dictonary.append(row['text'])
#imagePath = [ os.path.join(imgDir,str(name)+'.jpeg') for name in labels ]
dataset = []
train_dictonary, train_labels = shuffle(dictonary,
                                          labels,
                                          random_state=1)
for name in train_labels:
    imgPath = os.path.join(imgDir,str(name)+'.jpeg')
    if os.path.exists(imgPath):
        imagePath.append(imgPath) 

imagePath_train, imagePath_val, dictonary_train, dictonary_val = train_test_split(imagePath,dictonary,
                                                                                  test_size=0.2,random_state=0)
print(len(imagePath),len(dictonary)) 
print(len(imagePath_train),len(imagePath_val)) 
print(len(dictonary_train),len(dictonary_val)) 


517 517
413 104
413 104


In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size,
                                                                oov_token="<unk>",
                                                                filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(dictonary)
tokenizer.word_index['<pad>'] = 0                                                                 
tokenizer.index_word[0] = '<pad>'
images = []
for image_path in imagePath_train[10:]:
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    images.append(img)
images =np.array(images)
print(images.shape)
#imges = tf.keras.applications.inception_v3.preprocess_input(tf.expand_dims(imges, 0))
#freature_extratcor= tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')
#img_features=freature_extratcor(imges)
#img_features = tf.reshape(img_features, (img_features.shape[0], img_features.shape[1]*img_features.shape[2]*img_features.shape[3]))


In [55]:
print(img_features.shape)

(1, 131072)


In [38]:
print(text)
text_seqs = tokenizer.texts_to_sequences([text])
#print(text_seqs)
text_vector=tf.keras.preprocessing.sequence.pad_sequences(text_seqs, padding='post',maxlen=25)
print(text_vector.shape)

03 QUARTER SLEEVE 1PULL ON 2KNEE LENGTH 3REGULAR FIT 4PLAIN 2KNEE LENGTH 6ROUND NECK 7UNLINED 8POLYESTER 9BLACK 5TUNICS 4.94
(1, 25)


In [52]:
sequence_1_input = tf.keras.layers.Input(shape=(25,), dtype='int32')
feature_2_input = tf.keras.layers.Input(shape=(8*8*2048,))
embedding = tf.keras.layers.Embedding(500, 256)
lstm_layer = tf.keras.layers.Bidirectional( tf.keras.layers.LSTM(
                            256, dropout=0.17, 
                            recurrent_dropout=0.01))

fully_connected_in=tf.keras.layers.Dense(256)
fully_connected_out = tf.nn.relu(fully_connected_in(feature_2_input))
lstm_input2=embedding(fully_connected_out)